In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations
import datetime
import os
import copy
import math
from concurrent.futures import ThreadPoolExecutor

writeToFile = True

foundGraphs = 0

currentGraphFile = 0

testInteger = 0

#This is the array of Graph6 outputs
outputLineArray = []

#We can also cache the Graph6 strings
graph6Cache = {}

#This is the array of Sage graphs to exclude
graphsToExclude = None

outputStream = None

imgDirectory = "subgraph free"


def initSubgraphs():
    global graphsToExclude
    
    graphsToExclude = []
    
    graphsToExclude.append(graphs.PathGraph(5))
    graphsToExclude.append(graphs.PathGraph(4).join(Graph(1)))


def removeSubstring(myStr, substring):
    output_string = ""
    str_list = myStr.split(substring)
    for element in str_list:
        output_string += element
    return output_string

def subdirectory(subdirectoryName, number):
    # Get the current directory path
    current_directory = os.path.dirname(os.path.abspath(__file__))


    # Create the subdirectory if it doesn't exist
    subdirectory_path = os.path.join(current_directory, subdirectoryName)
    os.makedirs(subdirectory_path, exist_ok=True)

    # Create the file within the subdirectory
    file_path = os.path.join(subdirectory_path, "classified graphs " + str(number) + ".txt")
    return file_path


def adjacencyDict(edges):
        # Build a dictionary where each edge is a key and its value is a set of neighboring edges
    adjacency_dict = {}
    for edge in edges:
        if edge[0] not in adjacency_dict:
            adjacency_dict[edge[0]] = set()
        if edge[1] not in adjacency_dict:
            adjacency_dict[edge[1]] = set()
        adjacency_dict[edge[0]].add(edge[1])
        adjacency_dict[edge[1]].add(edge[0])
    
    #print(adjacency_dict)
    return adjacency_dict

def remove_node(adj_dict, node):
    # create a copy of the adjacency dictionary to modify
    new_adj_dict = copy.deepcopy(adj_dict)

    # remove the node and its edges from the dictionary
    
    #This takes care of the node
    del new_adj_dict[node]
    
    #This loops through the edges
    for neighbor in adj_dict[node]:
        
        getter = new_adj_dict.get(neighbor, None)
        if (getter != None) :
            if (node in new_adj_dict[neighbor]):
                new_adj_dict[neighbor].remove(node)
    # del new_adj_dict[node]

  # shift down the node indices higher than the removed node
    for i in range(node+1, max(new_adj_dict)+1):
        if i in new_adj_dict:
            new_adj_dict[i-1] = new_adj_dict.pop(i)
            new_adj_dict[i-1] = {j-1 if j>node else j for j in new_adj_dict[i-1]}
  

    return new_adj_dict


#This returns whether a SAGE graph has an induced subgraph B anywhere
def isSubgraphFree(G, B):
    sub_search = G.subgraph_search(B, induced=True)

    # Return True or False
    return (sub_search == None)

#This returns whether a SAGE graph has any of the graphs we don't want anywhere
def isSubgraphsFree(G):
    global graphsToExclude
    
    for B in graphsToExclude:
        subFree = isSubgraphFree(G, B)
        
        if (not subFree):
            return False
    

    return True


def graph_to_graph6_3(adj_dict):
    
    dictString = str(adj_dict)
    
    #This cache can get big, so we only do it for small graphs
    if (len(adj_dict) <= 5):
        getter = graph6Cache.get(dictString, None)
        if (getter != None):
                return getter
    
    # Create a graph object from the adjacency dictionary
    G = nx.Graph(adj_dict)

    # Obtain a Graph6 string from the graph object
    graph6 = nx.to_graph6_bytes(G).decode('ascii')

    # Remove the newline character at the end of the string
    graph6 = graph6.rstrip('\n')
    
    if (len(adj_dict) <= 5):
        graph6Cache[dictString] = graph6
    
    return graph6



def showGraph(dict_):
    print(dict_)
    
    G = nx.Graph(dict_)
                        
    color_array = ["red", "blue", "green", "gold", "orange", "purple", "teal", "black", "grey", "steelblue", "magenta", "violet", "dodgerblue", "brown"]
    color_array_trunc = []
    
    for i in range(len(dict_)):
        color_array_trunc.append(color_array[i])
        
    # Create a dictionary that maps each node to its color
    # color_map = {node: color_array[coloring[node]] for node in adjacency_dict.keys()}

    # Draw the colored graph using networkx
    nx.draw(G, with_labels=True, node_color=color_array_trunc, font_color='w')
    plt.show()

def classifyGraph(graph):
  
    if (isSubgraphsFree(graph)):
        # print("Critical graph found!")
        return [True, "The graph does not contain the subgraphs", graph]
    
    return [False, "The graph does contain one of the subgraphs. No good", graph]


def multithreading(min_nodes, max_nodes, num_threads):

    global imgDirectory
    global currentGraphFile
    global foundGraphs
    global outputStream
    
    if (not os.path.exists(imgDirectory)):
        os.makedirs(imgDirectory)
    
    initSubgraphs()
    
    # Create a thread pool executor with the specified number of threads

    with ThreadPoolExecutor(max_workers=num_threads) as executor:

        # Submit each number to the executor for processing

        futures = []

        path = os.path.join(imgDirectory, f"graph_{currentGraphFile}.g6")
        outputStream = open(path, "w")
        
        for num_nodes in range(min_nodes, max_nodes+1):
            nauty_generator = graphs.nauty_geng(str(num_nodes) + " -c")
            graphs_list = list(nauty_generator)

            for g in graphs_list:
                futures.append(executor.submit(foo,g))

        # Get the results from each future as they become available

        results = [future.result() for future in futures]
        
        # for r in results:
        #    foo2(r)
    
    saveFiles()
    outputStream.close()
    
    return


def foo(graph):
    
    global imgDirectory
    global currentGraphFile
    global foundGraphs
    global outputLineArray
    
    # This array has True in index 0 if the graph is P5-free
    # It ha a print message in index 1
    # The graph itself is in index 2
    graphResults = classifyGraph(graph)
    xGraph = graph.networkx_graph()
    
    if (graphResults[0]):
        
        adj_dict = adjacencyDict(xGraph.edges())
        
        graph6Str = graph_to_graph6_3(adj_dict)
        graph6Str = removeSubstring(graph6Str, ">>graph6<<")
        
        
        global writeToFile
        if (writeToFile):
            outputLineArray.append([graph6Str, xGraph])
    
    return graphResults


def saveFiles():
    
    global writeToFile
    global outputLineArray
    global currentGraphFile
    global foundGraphs
    global outputStream
    
    if (writeToFile):
        
        print(f"{len(outputLineArray)} graphs found!")

        for outputLine in outputLineArray:
            
            #showGraph(outputLine[1])
            
            outputStream.write(outputLine[0])
            outputStream.write("\n")

            foundGraphs = foundGraphs + 1
    
            if (foundGraphs > 500):
                foundGraphs = 0
                
                outputStream.close()

                print("File " + str(currentGraphFile) + " Created!")
                currentGraphFile = currentGraphFile + 1

                path = os.path.join(imgDirectory, f"graph_{currentGraphFile}.g6")
                outputStream = open(path, "w")
    
        
# Example usage

min_nodes = int(input("Enter the min number of nodes: "))
max_nodes = int(input("Enter the max number of nodes: "))
threads = int(input("Enter the number of threads: "))



print("Starting!")
start_time = datetime.datetime.now()

multithreading(min_nodes, max_nodes, threads)

end_time = datetime.datetime.now()

elapsed_time = end_time - start_time
totalSeconds = elapsed_time.total_seconds()



hours, remainder = divmod(int(totalSeconds), 3600)
minutes, seconds = divmod(remainder, 60)
roundSeconds = float("{:.2f}".format(math.modf(totalSeconds)[0] + seconds))

print("Elapsed time: {} hours, {} minutes, {} seconds".format(int(hours), int(minutes), roundSeconds))
print(testInteger)

Enter the min number of nodes: 4
Enter the max number of nodes: 8
Enter the number of threads: 3
Starting!
